In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS

In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
from data.loaders import TieredImagenet
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE, ConvBase, TADCEncoder
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy
from src.utils2 import Profiler

In [11]:
model_path = '../logs/DELPO_miniimagenet_5-way_5-shot_10-queries/exp8-eaen'
model_name = 'model_36000.pt'
learner = torch.load('{}/{}'.format(model_path, model_name))
learner = learner.to('cuda')

In [15]:
learner.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[-4.6589e-02,  8.5306e-02,  1.5025e-01],
                        [ 2.3872e-02, -8.0855e-02,  9.8225e-02],
                        [ 1.1540e-01, -1.2838e-02,  1.4685e-01]],
              
                       [[-6.3635e-04,  1.6862e-01, -1.8315e-01],
                        [ 1.2694e-01,  9.7512e-02, -9.9714e-02],
                        [-8.5788e-02, -4.4815e-02,  1.3261e-01]],
              
                       [[ 1.2149e-04, -1.8649e-01, -9.2558e-02],
                        [-8.4585e-03, -8.8237e-02,  1.2068e-01],
                        [-1.6378e-01,  8.5219e-02, -1.4752e-01]]],
              
              
                      [[[ 1.8163e-01, -1.7215e-02, -1.6119e-02],
                        [ 1.5310e-01,  6.3453e-02,  1.4108e-01],
                        [-1.8021e-01,  7.6058e-03, -1.9771e-01]],
              
                       [[-4.3892e-02, -1.6445e-02,  1.1611e-01],
                        [ 1.29

In [ ]:
train_dataset = l2l.vision.datasets.TieredImagenet(root='../../data', mode='train', download=True)
train_dataset = l2l.data.MetaDataset(train_dataset)

In [5]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 15
dataset = 'cifarfs'
root = '../../dataset/cifarfs'
order = False
inner_lr = 0.001
meta_lr = 0.001
reconst_loss = nn.MSELoss(reduction='none')
inner_adapt_steps_train = 5
meta_batch_size = 10
download = False

In [6]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'cifarfs'

args = Args(100, False, 0.1, 1, 1)

In [7]:
image_trans = transforms.Compose([transforms.ToTensor()])
train_tasks = gen_tasks(dataset, root, image_transforms=image_trans, download=download, mode='train',
                        n_ways=n_ways, k_shots=k_shots, q_shots=q_shots)
valid_tasks = gen_tasks(dataset, root, image_transforms=image_trans, download=download, mode='validation',
                        n_ways=n_ways, k_shots=k_shots, q_shots=q_shots)
test_tasks = gen_tasks(dataset, root, image_transforms=image_trans, download=download, mode='test',
                        n_ways=n_ways, k_shots=k_shots, q_shots=q_shots, num_tasks=600)

In [8]:
input = train_tasks.sample()[0]
input.shape

torch.Size([100, 3, 32, 32])

In [10]:
x = nn.Conv2d(3, 64, kernel_size=3, padding=1)(input)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 16, 16])

In [11]:
x = nn.Conv2d(64, 64, kernel_size=3, padding=1)(x)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 8, 8])

In [12]:
x = nn.Conv2d(64, 64, kernel_size=3, padding=1)(x)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 4, 4])

In [13]:
x = nn.Conv2d(64, 64, kernel_size=3, padding=1)(x)
x = nn.BatchNorm2d(64)(x)
x = nn.MaxPool2d(2)(x)
x.shape

torch.Size([100, 64, 2, 2])

In [14]:
learner = CCVAE(in_channels=3, base_channels=64, n_ways=n_ways, dataset='cifarfs', task_adapt=True, task_adapt_fn='eaen', args=args)

AttributeError: 'Args' object has no attribute 'n_ways'